In [1]:
%load_ext autoreload
%autoreload 2
import sys
import torch
from torch import nn
sys.path.append('..')
# sys.path.append('/system/user/beck/pwbeck/projects/regularization/ml_utilities')
from pathlib import Path
from typing import Union
from ml_utilities.torch_models.base_model import BaseModel
from ml_utilities.torch_models.fc import FC
from ml_utilities.torch_models import get_model_class
from omegaconf import OmegaConf

from erank.utils import load_directions_matrix_from_task_sweep
from erank.models.resnet import create_resnet
import matplotlib.pyplot as plt
gpu_id = 0

/home/max/anaconda3/envs/erank/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# config for resnet18-imagenet
img_size = 32
in_channels = 3
resnet_config = f"""
model:
  in_channels: {in_channels}
  act_fn: relu
  residual_option: B
  input_layer_config:
      kernel_size: 7
      batch_norm: true
      stride: 2
      padding: 6
      max_pool_kernel_size: 3
      max_pool_stride: 2
      max_pool_padding: 1
  resnet_blocks_config:
    - in_channels: 64
      out_channels: 64
      num_residual_blocks: 2
    - in_channels: 64
      out_channels: 128
      num_residual_blocks: 2
    - in_channels: 128
      out_channels: 256
      num_residual_blocks: 2
    - in_channels: 256
      out_channels: 512
      num_residual_blocks: 2
  linear_output_units:
    - 10
"""

In [3]:
resnet_config = OmegaConf.create(resnet_config)

In [4]:
resnet18 = create_resnet(**resnet_config.model)

In [5]:
image_batch = torch.randn(16, 3, 224, 224)
out_batch = resnet18(image_batch)
out_batch.shape

torch.Size([16, 10])

In [6]:
from torchsummary import summary
print(summary(resnet18, image_batch))

/home/max/.local/lib/python3.8/site-packages/torch/cuda/__init__.py:80: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at  ../c10/cuda/CUDAFunctions.cpp:112.)
  return torch._C._cuda_getDeviceCount() > 0


Layer (type:depth-idx)                   Output Shape              Param #
├─Sequential: 1-1                        [-1, 64, 58, 58]          --
|    └─Conv2d: 2-1                       [-1, 64, 115, 115]        9,472
|    └─BatchNorm2d: 2-2                  [-1, 64, 115, 115]        128
|    └─ReLU: 2-3                         [-1, 64, 115, 115]        --
|    └─MaxPool2d: 2-4                    [-1, 64, 58, 58]          --
├─Sequential: 1-2                        [-1, 512, 8, 8]           --
|    └─Sequential: 2-5                   [-1, 64, 58, 58]          --
|    |    └─_ResidualBlock: 3-1          [-1, 64, 58, 58]          74,112
|    |    └─_ResidualBlock: 3-2          [-1, 64, 58, 58]          74,112
|    └─Sequential: 2-6                   [-1, 128, 29, 29]         --
|    |    └─_ResidualBlock: 3-3          [-1, 128, 29, 29]         230,528
|    |    └─_ResidualBlock: 3-4          [-1, 128, 29, 29]         295,680
|    └─Sequential: 2-7                   [-1, 256, 15, 15]     